In [2]:
import torch.nn as nn
from torch.nn.functional import cross_entropy, softmax, relu
import numpy as np
import torch
from torch.utils import data
import utils
from torch.utils.data import DataLoader
import argparse

In [3]:
MAX_LEN = 11

In [4]:
class MultiHead(nn.Module):
    def __init__(self, n_head, model_dim, drop_rate):
        """
        :param n_head: 注意力头的数量，它指定了多头注意力中的子注意力头的个数
        :param model_dim: 模型维度，即输入特征或输出特征的维度（多头注意力中，输入特征和输出特征的维度相等）
        :param drop_rate: 丢弃率，它指定了在模型训练过程中应用于输出的丢弃比例，用于防止过拟合
        """
        super(MultiHead, self).__init__()
        self.head_dim = model_dim // n_head  #每个注意力头的维度
        self.n_head = n_head
        self.model_dim = model_dim
        # 下边3个线性层将输入特征进行线性投影，将模型维度转换为n_head * head_dim
        self.wq = nn.Linear(model_dim, n_head * self.head_dim)
        self.wk = nn.Linear(model_dim, n_head * self.head_dim)
        self.wv = nn.Linear(model_dim, n_head * self.head_dim)

        # 将多头注意力的输出进行线性变换，将模型维度保持为原始的model_dim
        self.o_dense = nn.Linear(model_dim, model_dim)
        self.o_drop = nn.Dropout(drop_rate)  #丢弃层，对输出进行随机丢弃，以防止过拟合
        self.layer_norm = nn.LayerNorm(model_dim)  #层归一化，对多头注意力的输出进行归一化处理
        self.attention = None  #初始化注意力权重为None，在前向传播过程中保存注意力权重供后续使用

    def forward(self, q, k, v, mask, training):
        # 残差连接
        residual = q  #保存输入 q 用于残差连接。它将在后面与输出相加
        dim_per_head = self.head_dim  #每一个注意力头的维度
        num_heads = self.n_head  #注意力头的数量
        batch_size = q.size(0)  #输入 q 的批量大小

        # 线性投影
        key = self.wk(k)  # [n, step, num_heads * head_dim]，step可以理解为句子的长度或单词
        value = self.wv(v)  # [n, step, num_heads * head_dim]
        query = self.wq(q)  # [n, step, num_heads * head_dim]

        # 按头数分割
        query = self.split_heads(query)  # [n, n_head, q_step, h_dim]
        key = self.split_heads(key)
        value = self.split_heads(value)  # [n, h, step, h_dim]

        # 缩放点积注意力，计算context上下文
        context = self.scaled_dot_product_attention(query, key, value, mask)  # [n, q_step, h*dv]，h：头数，dv：每个头的维度
        o = self.o_dense(context)  # [n, step, dim]
        o = self.o_drop(o)

        # 将残差连接的结果 residual 与输出 o 相加，并通过 self.layer_norm 进行层归一化处理
        o = self.layer_norm(residual + o)
        return o

    def split_heads(self, x):
        # 输入x张量的形状：[batch_size, seq_len, num_heads * head_dim]
        x = torch.reshape(x, (x.shape[0], x.shape[1], self.n_head, self.head_dim))
        return x.permute(0, 2, 1, 3)  #输出x张量的形状：[batch_size, num_heads, seq_len, head_dim]

    def scaled_dot_product_attention(self, q, k, v, mask=None):
        dk = torch.tensor(k.shape[-1]).type(torch.float)  #计算变量dk，它表示注意力机制中的缩放因子
        score = torch.matmul(q, k.permute(0, 1, 3, 2)) / (torch.sqrt(dk) + 1e-8)  # 计算注意力得分，[n, n_head, step, step]
        if mask is not None:
            # 如果存在掩码张量，则进行下一步操作。掩码用于屏蔽注意力得分中某些位置，以使得在 softmax 操作之后这些位置的注意力权重接近于 0
            score = score.masked_fill_(mask, -np.inf)  #将掩码位置的值更改为负无穷，以实现屏蔽效果。注意，这里使用了原地操作 masked_fill_
        self.attention = softmax(score, dim=-1)  #对注意力得分进行 softmax 操作，得到注意力权重
        context = torch.matmul(self.attention, v)  # 通过矩阵乘法计算上下文向量，[n, num_head, step, head_dim]
        context = context.permute(0, 2, 1, 3)  # [n, step, num_head, head_dim]
        context = context.reshape((context.shape[0], context.shape[1],
                                   -1))  #将上下文向量进行维度重塑，将注意力头的数量和特征维度合并到一个维度中，得到形状为 [n, step, model_dim] 的上下文张量
        return context  # [n, step, model_dim]

在这个上下文中，"step" 表示序列的长度或时间步的数量。在自然语言处理中，可以将它理解为句子的长度或单词的数量。因此，形状 `[n, step, num_heads * head_dim]` 表示有 `n` 个样本/实例，每个样本包含一个长度为 `step` 的序列，每个序列经过线性投影后的特征维度为 `num_heads * head_dim`。

# 原地操作

在PyTorch中，原地操作（in-place operation）是指直接在原始张量上进行修改，而不创建新的张量。原地操作可以通过在函数名后添加下划线（`_`）来表示。

在给定的代码中，`score.masked_fill_(mask, -np.inf)`使用了原地操作 `masked_fill_`。它将张量 `score` 中与掩码 `mask` 对应位置的值更改为负无穷（`-np.inf`），从而实现对这些位置的屏蔽效果。这意味着在计算注意力权重时，被掩码的位置的注意力得分会变为负无穷，经过 softmax 操作后会接近于零。原地操作可以节省内存并减少不必要的张量复制。

不是所有函数名后面都可以通过加下划线来实现原地操作。只有在函数的实现中明确指定支持原地操作的情况下，才可以在函数名后面加下划线实现原地操作。

在 PyTorch 中，只有一部分函数和方法支持原地操作。通常，这些函数和方法的命名规则是函数名或方法名后面带有下划线（`_`），例如`masked_fill_()`、`add_()`、`mul_()`等。这些函数会直接在原始张量上进行修改，而不创建新的张量。

需要注意的是，原地操作可能会改变原始张量的内容，因此在使用时需要小心。一般建议在不需要保留原始张量内容的情况下才使用原地操作。如果需要保留原始张量并进行修改，可以使用非原地操作或先创建一个副本进行操作。

In [5]:
class PositionWiseFFN(nn.Module):
    """实现位置编码层的类，用于在Transformer模型中进行位置编码和前馈神经网络的操作"""

    def __init__(self, model_dim, dropout=0.0):
        """
        :param model_dim: 模型的维度（特征维度）
        :param dropout: 丢弃率
        """
        super().__init__()
        dff = model_dim * 4  #计算前馈神经网络中隐藏层的维度
        self.l = nn.Linear(model_dim, dff)  #线性层，用于进行维度变换，将输入特征维度变换为前馈神经网络隐藏层的维度
        self.o = nn.Linear(dff, model_dim)  #线性层，用于将前馈神经网络隐藏层的输出维度变换回输入特征维度
        self.dropout = nn.Dropout(dropout)  #在前馈神经网络的输出上应用Dropout操作，以防止过拟合
        self.layer_norm = nn.LayerNorm(model_dim)  #对输入进行归一化处理

    def forward(self, x):
        o = relu(self.l(x))  #通过线性层self.l对输入进行变换，并通过ReLU激活函数进行非线性处理
        o = self.o(o)  #再次通过线性层self.o对隐藏层的输出进行变换
        o = self.dropout(o)  #随机丢弃一部分隐藏层的输出

        o = self.layer_norm(x + o)  #将原始输入张量x与经过变换和Dropout后的输出张量相加，并通过Layer Normalization进行归一化处理
        return o  # [n, step, dim]

PositionWiseFFN类并没有实现位置编码。它是一个位置智能的前馈神经网络模块，用于对输入特征进行非线性变换和维度映射。位置编码通常是一种用于为序列数据引入位置信息的技术，在Transformer等模型中常用于对序列中每个位置的特征进行编码。

PositionWiseFFN类中的前向传播函数将输入特征进行线性变换、激活函数处理、维度映射等操作，并通过残差连接和层归一化来保持输入和输出特征的一致性。它并没有显式地包含位置编码的部分。如果要在Transformer模型中使用位置编码，通常会在模型的输入层或者编码器模块中添加位置编码的步骤。

# 前馈神经网络[Q/A]

**Q：什么是前馈神经网络？就是指网络的前向传播嘛？**

"前馈神经网络"（Feedforward Neural Network）通常指的是具有多个隐藏层的神经网络模型，其中信息从输入层经过一系列的前向传播操作传递到输出层。前馈神经网络是最基本的神经网络类型，也是最常见的神经网络架构之一。

在前馈神经网络中，信息在网络中单向流动，从输入层经过一系列的隐藏层，最终到达输出层。每个隐藏层通常包含一些线性变换和非线性激活函数，以便引入非线性性质并增加模型的表达能力。隐藏层之间的连接权重通过训练过程中的反向传播算法进行学习。

前馈神经网络在许多任务和领域中都得到广泛应用，如图像分类、自然语言处理、语音识别等。它们是深度学习的基础，并在许多深度学习模型和架构中发挥着重要作用。

**Q：那它跟某个类中的forward函数有什么区别？**

前馈神经网络（Feedforward Neural Network）是一种神经网络架构，它由多个层组成，每一层的输出作为下一层的输入，信息在网络中单向传播。网络的前向传播是指输入数据通过网络的各个层进行计算，最终得到输出结果的过程。

而类中的`forward`函数是定义在类中的方法，用于定义输入数据在该类模型中的前向传播过程。在神经网络模型中，`forward`函数通常包括对输入数据的一系列操作，如线性变换、激活函数、池化等，以及对这些操作结果的组合和处理，最终得到输出结果。

因此，前馈神经网络是一种整体的网络结构，包含多个层和参数，而类中的`forward`函数是对输入数据进行具体计算的方法。`forward`函数定义了模型的前向传播逻辑，它在训练和推断过程中被调用，用于对输入数据进行处理并生成输出结果。

**Q：那前馈神经网络跟nn.Sequential()有什么区别呢？**

前馈神经网络和`nn.Sequential()`是两种不同的概念。

前馈神经网络（Feedforward Neural Network）是一种神经网络架构，由多个层组成，每一层的输出作为下一层的输入，信息在网络中单向传播。前馈神经网络可以具有不同的结构，包括全连接层、卷积层、循环层等，用于处理不同类型的数据和任务。

`nn.Sequential()`是PyTorch中提供的一个方便的模型容器，用于按照顺序组合多个层或模块。它可以将多个层按照定义的顺序依次连接起来，形成一个前馈的网络结构。`nn.Sequential()`可以简化模型的定义过程，通过传入一系列层或模块的实例，自动按照它们在列表中的顺序进行前向传播。

区别在于，前馈神经网络是一种网络架构的概念，而`nn.Sequential()`是PyTorch提供的一个工具，用于方便地定义和组合网络的层或模块。`nn.Sequential()`可以作为一种实现前馈神经网络的方式之一，通过它可以按照顺序定义和连接网络的各个层，但并不是唯一的方式，还可以使用其他方式手动定义和连接网络的层。

**Q：是不是任意一个含有forward函数的类，都可以看作是某一种前馈神经网络？**

不完全正确。含有`forward`函数的类可以被看作是一种模型或网络，但不一定都是前馈神经网络。

前馈神经网络是指信息在网络中单向传播的网络结构，每一层的输出作为下一层的输入，没有反馈连接。这种网络结构可以由多个层组成，每一层的计算依赖于前一层的输出。前馈神经网络常见的形式包括全连接神经网络（MLP）和卷积神经网络（CNN）等。

含有`forward`函数的类可以实现各种不同的网络结构和计算方式，不一定都是前馈神经网络。例如，循环神经网络（RNN）和注意力机制（Attention）等网络结构在计算过程中具有反馈或跨时间步的连接，不符合前馈神经网络的定义。

因此，虽然含有`forward`函数的类可以表示一种模型或网络，但要将其看作前馈神经网络，还需满足单向传播且没有反馈连接的特性。

**Q：所以前馈神经网络指的是某一类网络，而非某个特定的网络？**

是的，前馈神经网络指的是一类网络结构，而不是指特定的某个网络。它是一种广义的概念，用于描述信息在网络中单向传播的特性，每一层的输出只作为下一层的输入，不存在反馈连接。

在深度学习中，前馈神经网络包括多个层的网络结构，每一层的计算依赖于前一层的输出。这种网络结构可以是全连接神经网络（MLP）、卷积神经网络（CNN）、残差网络（ResNet）等。这些网络都满足了前馈神经网络的定义，但在具体的架构和设计上可能有所差异。

因此，前馈神经网络是一类网络结构的总称，表示信息在网络中单向传播，没有反馈连接的特性，而具体的网络架构则是这一类网络结构的具体实现。

In [6]:
class EncoderLayer(nn.Module):
    """编码器各层（multihead+ffn）"""

    def __init__(self, n_head, emb_dim, drop_rate):
        super().__init__()
        self.mh = MultiHead(n_head, emb_dim, drop_rate)
        self.ffn = PositionWiseFFN(emb_dim, drop_rate)

    def forward(self, xz, training, mask):
        # xz: [n, step, emb_dim]
        context = self.mh(xz, xz, xz, mask, training)  # [n, step, emb_dim]
        o = self.ffn(context)
        return o

In [7]:
class Encoder(nn.Module):
    """编码器，将前面定义的编码器各层堆叠起来"""

    def __init__(self, n_head, emb_dim, drop_rate, n_layer):
        super().__init__()
        self.encoder_layers = nn.ModuleList(
            [EncoderLayer(n_head, emb_dim, drop_rate) for _ in range(n_layer)]
        )

    def forward(self, xz, training, mask):
        for encoder in self.encoder_layers:
            xz = encoder(xz, training, mask)
        return xz  # [n, step, emb_dim]

In [8]:
class DecoderLayer(nn.Module):
    def __init__(self, n_head, model_dim, drop_rate):
        super().__init__()
        self.mh = nn.ModuleList([MultiHead(n_head, model_dim, drop_rate) for _ in range(2)])
        self.ffn = PositionWiseFFN(model_dim, drop_rate)

    def forward(self, yz, xz, training, yz_look_ahead_mask, xz_pad_mask):
        dec_output = self.mh[0](yz, yz, yz, yz_look_ahead_mask, training)  # [n, step, model_dim]
        dec_output = self.mh[1](dec_output, xz, xz, xz_pad_mask, training)  # [n, step, model_dim]
        dec_output = self.ffn(dec_output)  # [n, step, model_dim]
        return dec_output

In [9]:
class Decoder(nn.Module):
    def __init__(self, n_head, model_dim, drop_rate, n_layer):
        super().__init__()

        self.num_layers = n_layer

        self.decoder_layers = nn.ModuleList(
            [DecoderLayer(n_head, model_dim, drop_rate) for _ in range(n_layer)]
        )

    def forward(self, yz, xz, training, yz_look_ahead_mask, xz_pad_mask):
        for decoder in self.decoder_layers:
            yz = decoder(yz, xz, training, yz_look_ahead_mask, xz_pad_mask)
        return yz  # [n, step, model_dim]

## nn.ModuleList V.S. nn.Sequential

nn.ModuleList和nn.Sequential都可以用于组合多个神经网络模块，但它们有一些区别。

nn.ModuleList是用于存储多个子模块的列表，它将其子模块作为属性存储在类中。与普通的Python列表不同，nn.ModuleList的子模块会自动添加到计算图中，在模型的前向传递时被调用。因此，当使用nn.ModuleList时，我们要明确每个子模块的名称和顺序，以便正确地访问和更新它们。

举个例子，假设我们有三个卷积层，我们可以使用nn.ModuleList将它们组合起来：

```python
import torch.nn as nn

class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet, self).__init__()
        self.conv_layers = nn.ModuleList([
            nn.Conv2d(3, 16, kernel_size=3, padding=1),
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.Conv2d(16, 32, kernel_size=3, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU()
        ])

    def forward(self, x):
        for layer in self.conv_layers:
            x = layer(x)
        return x
```

这里使用nn.ModuleList将三个卷积层组合在一起，并在前向传递中遍历它们。请注意，在nn.ModuleList中，我们不需要手动指定每个层的名称。

相比之下，nn.Sequential则是一个按顺序执行的容器，其中每个子模块都被称为“层”，并且可以通过索引来访问它们。在创建nn.Sequential对象时，我们只需要按照顺序将所有的层传递给它即可。与nn.ModuleList不同，我们不需要为每个层命名，并且在前向传递中，nn.Sequential会自动按照顺序执行所有的层。

举个例子，上面的ConvNet也可以使用nn.Sequential实现：

```python
import torch.nn as nn

class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet, self).__init__()
        self.conv_layers = nn.Sequential(
            nn.Conv2d(3, 16, kernel_size=3, padding=1),
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.Conv2d(16, 32, kernel_size=3, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU()
        )

    def forward(self, x):
        return self.conv_layers(x)
```

这里使用nn.Sequential将三个卷积层组合在一起，并在前向传递中按顺序执行它们。请注意，在nn.Sequential中，我们无需手动指定每个层的名称，但必须使用索引来访问它们。

总的来说，nn.ModuleList适合用于需要动态添加或删除子模块的情况，而nn.Sequential则适合用于按顺序执行固定数量的子模块的情况。

In [10]:
class PositionEmbedding(nn.Module):
    def __init__(self, max_len, emb_dim, n_vocab):
        """
        将输入序列的索引映射为嵌入向量，并为每个位置添加位置编码，以帮助模型学习序列中的位置信息
        :param max_len: 序列的最大长度
        :param emb_dim: 位置嵌入向量的维度
        :param n_vocab: 词汇表的大小
        """
        super().__init__()
        pos = np.expand_dims(np.arange(max_len), 1)  # 创建一个形状为(max_len, 1)的numpy数组，其中包含从0到max_len-1的序列。这个数组表示每个位置的索引
        pe = pos / np.power(1000, 2 * np.expand_dims(np.arange(emb_dim) // 2,
                                                     0) / emb_dim)  # 根据位置索引和嵌入维度计算位置编码。通过对位置索引进行数学运算，生成一个形状为(max_len, emb_dim)的numpy数组，其中包含了位置编码信息
        pe[:, 0::2] = np.sin(pe[:, 0::2])  #将位置编码数组的偶数列进行正弦函数变换
        pe[:, 1::2] = np.cos(pe[:, 1::2])  #将位置编码数组的奇数列进行余弦函数变换
        pe = np.expand_dims(pe, 0)  # 在位置编码数组的第一个维度上添加一个维度，使其变为形状为(1, max_len, emb_dim)的数组
        self.pe = torch.from_numpy(pe).type(torch.float32)  #将位置编码数组转换为PyTorch的张量，并将数据类型设置为float32
        self.embeddings = nn.Embedding(n_vocab, emb_dim)  #将输入序列的索引映射为嵌入向量
        self.embeddings.weight.data.normal_(0, 0.1)  #初始化嵌入层的权重，使用正态分布随机初始化，均值为0，标准差为0.1

    def forward(self, x):
        device = self.embeddings.weight.device  #获取嵌入层的设备信息
        self.pe = self.pe.to(device)
        x_embed = self.embeddings(x) + self.pe  # 通过嵌入层将输入序列的索引映射为嵌入向量，并与位置编码张量相加，[n, step, emb_dim]
        return x_embed  # 返回嵌入后的序列[n, step, emb_dim]（批次大小，序列长度，训练向量的维度）

# 位置编码

位置编码的设计有以下几个巧妙之处：

1. **可学习性和可变性：** 位置编码并不是固定的，而是通过数学函数计算得到的。这样设计的好处是，模型可以根据训练数据自动学习到不同位置的编码，并根据任务的需要进行调整和适应。

2. **连续性和周期性：** 位置编码的设计采用了正弦和余弦函数，使得编码具有连续性和周期性的特点。这样可以捕捉到序列中位置之间的关系和周期性的模式，有助于模型学习序列中不同位置的相对位置信息。

3. **维度交替：** 位置编码中的正弦函数和余弦函数被交替应用于不同的维度，使得每个维度的编码具有不同的周期性。这种设计可以增加编码的多样性，提供更丰富的位置表示。

4. **可解释性和可视化：** 位置编码的设计使得每个位置都有一个唯一的编码向量，可以将其可视化并解释为模型对不同位置的关注程度或权重。这样有助于理解模型在处理序列任务时对位置信息的利用情况。

通过这些巧妙的设计，位置编码在自然语言处理和序列建模任务中广泛应用，有效地提供了位置信息的表示和建模能力，有助于提升模型在处理序列数据时的性能。

In [11]:
class Transformer(nn.Module):
    def __init__(self, n_vocab, max_len, n_layer=6, emb_dim=512, n_head=8, drop_rate=0.1, padding_idx=0):
        super(Transformer, self).__init__()
        self.max_len = max_len  #最大序列长度
        self.padding_idx = torch.tensor(padding_idx)  #填充标记的索引
        self.dec_v_emb = n_vocab  #解码器的词汇量大小

        self.embed = PositionEmbedding(max_len, emb_dim, n_vocab)  #位置编码的嵌入层
        self.encoder = Encoder(n_head, emb_dim, drop_rate, n_layer)  #编码器
        self.decoder = Decoder(n_head, emb_dim, drop_rate, n_layer)  #解码器
        self.o = nn.Linear(emb_dim, n_vocab)  #输出层
        self.opt = torch.optim.Adam(self.parameters(), lr=0.002)  #优化器

    def forward(self, x, y, training=None):
        x_embed, y_embed = self.embed(x), self.embed(y)  #输入序列的嵌入表示
        pad_mask = self._pad_mask(x)  #填充屏蔽，指示输入序列 x 中填充标记的位置
        encoder_z = self.encoder(x_embed, training,
                                 pad_mask)  #应用于输入序列 x_embed 的编码器模块的输出。编码器模块处理输入序列并生成输入的表示，称为 encoder_z
        yz_look_ahead_mask = self._look_ahead_mask(y)  #前瞻屏蔽，屏蔽了解码器输入序列 y 中的未来位置
        decoded_z = self.decoder(y_embed, encoder_z, training, yz_look_ahead_mask,
                                 pad_mask)  #输入序列 y_embed 和编码器输出 encoder_z 的解码器模块的输出。解码器模块生成基于输入序列条件的输出序列的表示
        o = self.o(decoded_z)  #线性层的输出，线性层将解码器表示映射到词汇表大小
        return o

    def step(self, x, y):
        self.opt.zero_grad()  #梯度清零
        logits = self(x, y[:, :-1], training=True)  #调用Transformer类（本类）中的forward方法来计算模型的预测输出
        pad_mask = ~torch.eq(y[:, 1:], self.padding_idx)  #创建一个布尔掩码，用于标识目标序列中不是填充标记的位置
        loss = cross_entropy(logits.reshape(-1, self.dec_v_emb), y[:, 1:].reshape(-1))  #损失函数
        loss.backward()  #反向传播
        self.opt.step()  #更新参数
        return loss.cpu().data.numpy(), logits  #返回loss值和预测的概率分布

    def _pad_bool(self, seqs):
        """生成一个布尔张量，表示序列中哪些位置是填充标记的位置。
        它接收一个形状为 (batch_size, seq_len) 的输入张量 seqs，并返回一个与 seqs 相同形状的布尔张量，
        其中填充标记的位置为 True，非填充标记的位置为 False"""
        o = torch.eq(seqs, self.padding_idx)
        return o

    def _pad_mask(self, seqs):
        """生成一个填充掩码张量，以屏蔽填充标记位置的信息。
        它接收一个形状为 (batch_size, seq_len) 的输入张量 seqs，
        并基于填充标记的位置生成一个形状为 (batch_size, 1, seq_len) 的掩码张量，
        其中填充标记的位置为 True，非填充标记的位置为 False。
        这个掩码张量在后续的计算中将被应用于对填充标记位置的屏蔽"""
        len_q = seqs.size(1)
        mask = self._pad_bool(seqs).unqueeze(1).expand(-1, len_q, -1)
        return mask.unsqueeze(1)

    def _look_ahead_mask(self, seqs):
        """生成一个前瞻掩码张量，以屏蔽解码器在生成序列时只能依赖前面的位置。
        它接收一个形状为 (batch_size, seq_len) 的输入张量 seqs，
        并生成一个形状为 (seq_len, seq_len) 的掩码张量，
        其中对角线上方的元素为 False，对角线及对角线以下的元素为 True。
        这个掩码张量在解码器的自注意力机制中使用，以确保在生成当前位置的输出时不会依赖后面的位置"""
        device = next(self.parameters()).device
        batch_size, seq_len = seqs.shape
        mask = torch.triu(torch.ones((seq_len, seq_len), dtype=torch.float.long), diagonal=1).to(device)
        mask = torch.where(self._pad_bool(seqs)[:, None, None, :], 1, mask[None, None, :, :]).to(device)
        return mask > 0

    def translate(self, src, v2i, i2v):
        self.eval()  #评估模式，用于推断过程
        device = next(self.parameters()).device  #获取模型参数所在的设备
        src_pad = src  #将源序列src赋值给 src_pad
        # 创建目标序列的张量 target，通过将 <GO> 标记重复拼接成 len(src) 行，然后使用 utils.pad_zero 函数对其进行填充，使其长度为 self.max_len+1。最后将其转换为张量并移动到指定的设备
        target = torch.from_numpy(
            utils.pad_zero(np.array([
                [v2i["<GO>"], ] for _ in range(len(src))]), self.max_len + 1)).to(device)
        x_embed = self.embed(src_pad)  #对源序列进行嵌入编码
        #  对嵌入向量进行编码，得到编码后的张量 encoded_z。这里传递了参数 False 表示不使用训练模式，以及 mask=self._pad_mask(src_pad) 表示使用填充掩码来屏蔽填充位置。
        encoded_z = self.encoder(x_embed, False, mask=self._pad_mask(src_pad))
        for i in range(0, self.max_len):
            y = target[:, :-1]  #将目标序列去除最后一个标记，得到输入序列 y
            y_embed = self.embed(y)
            decoded_z = self.decoder(y_embed, encoded_z, False, self._look_ahead_mask(y), self._pad_mask(src_pad))
            o = self.o(decoded_z)[:, i, :]
            idx = o.argmax(dim=1).detach()  #获取输出向量 o 在维度 1 上的最大值索引，即预测的下一个标记的索引。通过 detach() 方法将其从计算图中分离
            target[:, i + 1] = idx  #更新解码器的输入来预测下一个位置
        self.train()
        return target

# 填充屏蔽和前瞻屏蔽

填充屏蔽是用于屏蔽填充标记的位置，这些填充标记通常表示输入序列中的无效或填充部分。它确保在计算注意力时，模型不会将注意力放在这些填充位置上。

具体来说，填充屏蔽通过将填充位置对应的注意力权重设置为负无穷（或者在 softmax 之前设置为 0）来实现。这样，在计算注意力得分后，填充位置的得分将变为负无穷或接近于零，从而在经过 softmax 激活后接近于 0。

前瞻屏蔽用于在解码器中，确保生成当前位置的输出时只依赖于当前位置及其之前的输入。这是因为在解码器生成输出序列时，为了遵循自然语言的生成顺序，模型不能依赖于后续的位置。

前瞻屏蔽通过在解码器的注意力计算中使用一个掩码矩阵，将当前位置之后的位置掩盖（将注意力权重设置为负无穷或 0）。这样，在计算注意力得分后，模型将无法在生成当前位置的输出时注意到后续的位置，从而实现前瞻屏蔽的效果。

## logits=self(x, y[:,:-1], training=True)的解释

在`step`函数中，`logits=self(x, y[:,:-1], training=True)`的意思是调用`Transformer`类的`forward`方法来计算模型的预测输出。

具体解释如下：
- `x` 是输入序列，`y` 是目标序列。
- `y[:,:-1]` 表示目标序列去除最后一个标记，因为在训练过程中，解码器的输入是目标序列的前缀，需要预测目标序列的下一个标记。
- `self(x, y[:,:-1], training=True)` 调用 `Transformer` 类的 `forward` 方法，将输入序列 `x` 和截断的目标序列 `y[:,:-1]` 作为输入，`training=True` 表示在训练模式下进行前向传播。
- `logits` 是模型在给定输入序列和目标序列前缀的情况下，预测输出序列的 logit 值。它表示模型对下一个标记的概率分布。

`logits` 的形状取决于模型的输出和目标序列的长度。假设模型的输出词汇表大小为 `n_vocab`，目标序列的长度为 `seq_len`（去除了最后一个标记），则 `logits` 的形状为 `(batch_size, seq_len, n_vocab)`。

具体地，在 `Transformer.step()` 函数中，通过调用 `self(x, y[:,:-1], training=True)` 获取 `logits`，其中：
- `x` 是输入序列的张量，形状为 `(batch_size, input_seq_len)`。
- `y` 是目标序列去除最后一个标记后的张量，形状为 `(batch_size, target_seq_len-1)`。

因此，`logits` 的形状为 `(batch_size, target_seq_len-1, n_vocab)`，表示对目标序列的预测结果。每个元素 `logits[i, j, k]` 表示第 `i` 个样本在第 `j` 个时间步预测的词汇表中索引为 `k` 的词的概率分布。

`torch.eq(input, other, out=None)`函数是 PyTorch 的一个逐元素比较函数，用于比较两个张量是否相等。

它将两个张量 `input` 和 `other` 逐元素进行比较，返回一个新的张量，新张量中每个元素都表示对应位置上 `input` 和 `other` 中的元素是否相等。如果相等，则对应位置上的元素为1，否则为0。

这个函数还支持广播机制，即两个张量形状不同但可以通过广播变成相同形状时，仍能正确比较并返回结果。

以下是使用 `torch.eq()` 函数的示例代码：

``` python
import torch

x = torch.tensor([1, 2, 3])
y = torch.tensor([1, 2, 4])
z = torch.tensor([1, 2])

# 比较 x 和 y 是否相等
result = torch.eq(x, y)
print(result)  # tensor([True, True, False])

# 比较 x 和 z 是否相等，因为形状不同无法广播，会提示错误
result = torch.eq(x, z)  # ValueError: operands have incompatible shapes
```

In [12]:
torch.eq(torch.tensor([[1, 2, 3], [4, 5, 6]]), torch.tensor([[3], [6]]))

tensor([[False, False,  True],
        [False, False,  True]])

In [16]:
def train(emb_dim=32, n_layer=3, n_head=4):
    dataset = utils.DateData(4000)
    print("Chinese time order: yy/mm/dd ", dataset.date_cn[:3], "\nEnglish time order: dd/M/yyyy", dataset.date_en[:3])
    print("Vocabularies: ", dataset.vocab)
    print(f"x index sample:  \n{dataset.idx2str(dataset.x[0])}\n{dataset.x[0]}",
          f"\ny index sample:  \n{dataset.idx2str(dataset.y[0])}\n{dataset.y[0]}")
    loader = DataLoader(dataset, batch_size=32, shuffle=True)
    model = Transformer(n_vocab=dataset.num_word, max_len=MAX_LEN, n_layer=n_layer, emb_dim=emb_dim, n_head=n_head,
                        drop_rate=0.1, padding_idx=0)
    # 如果gpu可用，则使用gpu
    if torch.cuda.is_available():
        print("GPU train avaliable")
        device = torch.device("cuda")
        model = model.cuda()
    else:
        device = torch.device("cpu")
        model = model.cpu()
    for i in range(100):  #进行 100 个 epochs 的训练
        for batch_idx, batch in enumerate(loader):  #遍历数据加载器中的每个批次数据
            bx, by, decoder_len = batch  #当前批次中的输入数据 bx、目标数据 by 和解码器长度 decoder_len 解包出来
            bx, by = torch.from_numpy(utils.pad_zero(bx, max_len=MAX_LEN)).type(torch.LongTensor).to(
                device), torch.from_numpy(utils.pad_zero(by, MAX_LEN + 1)).type(torch.LongTensor).to(
                device)  #将输入和目标数据进行填充，并转换为对应的张量类型，并将它们移动到设备上（GPU 或 CPU）进行计算
            loss, logits = model.step(bx, by)  #计算损失和预测结果
            if batch_idx % 50 == 0:
                target = dataset.idx2str(by[0, 1:-1].cpu().data.numpy())  #将目标序列转换为文本形式，并赋值给 target
                pred = model.translate(bx[0:1], dataset.v2i, dataset.i2v)  #使用模型进行翻译，传入一个样本 bx[0:1]，并使用数据集的词汇表进行索引转换
                res = dataset.idx2str(pred[0].cpu().data.numpy())  #将翻译结果转换为文本形式，并赋值给 res
                src = dataset.idx2str(bx[0].cpu().data.numpy())  #将输入序列转换为文本形式，并赋值给 src
                print(
                    "Epoch: ", i,
                    "| t: ", batch_idx,
                    "| loss: %.3f" % loss,
                    "| input: ", src,
                    "| target: ", target,
                    "| inference: ", res,
                )

In [17]:
if __name__ == "__main__":
    parser = argparse.ArgumentParser()
    parser.add_argument("--emb_dim", type=int, help="change the model dimension")
    parser.add_argument("--n_layer", type=int, help="change the number of layers in Encoder and Decoder")
    parser.add_argument("--n_head", type=int, help="change the number of heads in MultiHeadAttention")

    args = parser.parse_args()
    args = dict(filter(lambda x: x[1], vars(args).items()))
    train(**args)

usage: ipykernel_launcher.py [-h] [--emb_dim EMB_DIM] [--n_layer N_LAYER]
                             [--n_head N_HEAD]
ipykernel_launcher.py: error: unrecognized arguments: -f C:\Users\cong\AppData\Roaming\jupyter\runtime\kernel-2b8d14a4-bd99-42ff-b5ed-1c439cc50f39.json


SystemExit: 2

In [15]:
% tb

SystemExit: 2